In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from fredapi import Fred



In [3]:
df = pd.read_csv("data.csv", sep=";", decimal=",")
df = df.rename(columns={
    "Column1": "Date",
    "Column2": "SPX",
    "Column3": "S5SFTW",
    "Column4": "S5PHRM",
    "Column5": "S5CPGS",
    "Column6": "S5ENRSX",
    "Column7": "S5FDBT",
    "Column8": "S5TECH",
    "Column9": "S5RETL",
    "Column10": "S5BANKX",
    "Column11": "S5HCES",
    "Column12": "S5DIVF",
    "Column13": "S5UTILX",
    "Column14": "S5MEDA",
    "Column15": "S5REAL",
    "Column16": "S5TELSX",
    "Column17": "S5MATRX",
    "Column18": "S5INSU",
    "Column19": "S5FDSR",
    "Column20": "S5HOUS",
    "Column21": "S5SSEQX",
    "Column22": "S5TRAN",
    "Column23": "S5HOTR",
    "Column24": "S5CODU",
    "Column25": "S5AUCO",
    "Column26": "S5COMS",
})
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")

In [4]:
def GetReturn(df,date,lookback):
    date=pd.to_datetime(date)
    if date not in df["Date"].values:#add breaker if windows not in df
        raise ValueError("Date not in dataframe")
    returns_df = df[["Date","S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS"]].copy()

    date_list=returns_df.drop(columns="Date")
    date_index = returns_df.index[returns_df["Date"] == date][0]
    returns_df=returns_df[(returns_df.index<=date_index) & (returns_df.index>=date_index-lookback) ]
    returns_df.drop(columns="Date",inplace=True)

    returns_df = np.log(returns_df/ returns_df.shift(1))
    returns_df.dropna(inplace=True)
    #print(returns_df.std().mean()) #verification if std is around 1% daily

    return returns_df


def GetReturnSPX(df,date,lookback):
    date=pd.to_datetime(date)
    if date not in df["Date"].values:#add breaker if windows not in df
        raise ValueError("Date not in dataframe")
    returns_df = df[["Date","SPX"]].copy()

    date_list=returns_df.drop(columns="Date")
    date_index = returns_df.index[returns_df["Date"] == date][0]
    returns_df=returns_df[(returns_df.index<=date_index) & (returns_df.index>=date_index-lookback) ]
    returns_df.drop(columns="Date",inplace=True)

    returns_df = np.log(returns_df/ returns_df.shift(1))
    returns_df.dropna(inplace=True)
    #print(returns_df.std().mean()) #verification if std is around 1% daily

    return returns_df

#Returns=GetReturn(df,"2020-05-11",lookback=180)
#ReturnsSPX=GetReturnSPX(df,"2020-05-11",lookback=180)

In [5]:
def GetSigma(df,date,lookback):
    returns_df=GetReturn(df,date,lookback=lookback)
    #covariance matric from returns_df
    sigma_windowed=returns_df.cov()

    return sigma_windowed

#Sigma=GetSigma(df,"2020-05-11",lookback=180)

In [6]:
def GetRfDataframe(df):
    fred = Fred(api_key="5c742a53d96bd3085e9199dcdb5af60b")
    riskfree = fred.get_series('DFF')
    # riskfree = fred.get_series('DTB1MO')

    riskfree = riskfree.to_frame(name='FedFunds')
    riskfree.index.name = "Date"
    riskfree = riskfree[riskfree.index >= "2002-01-01"]
    riskfree["FedFunds"]=riskfree["FedFunds"]/100
    list_days_open = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")
    list_days_full = pd.to_datetime(riskfree.index, dayfirst=True, errors="coerce")

    list_days_open=[pd.to_datetime(date) for date in list_days_open]
    list_days_full=[pd.to_datetime(date) for date in list_days_full]


    list_days_open_pondered=[]
    riskfree_list=[]
    count_list=[]
    timestamp=0
    while timestamp < len(list_days_full)-1:

      if list_days_full[timestamp+1] in list_days_open:
            list_days_open_pondered.append(list_days_full[timestamp])
            riskfree_list.append(riskfree["FedFunds"].loc[list_days_full[timestamp]])
            count_list.append(1)
            timestamp += 1

      else:
          count = 0
          timestampbis = timestamp
          while (timestamp + 1 < len(list_days_full)) and (list_days_full[timestamp + 1] not in list_days_open):
              timestamp += 1
              count += 1

          list_days_open_pondered.append(list_days_full[timestampbis])  # jour de départ
          riskfree_list.append(riskfree["FedFunds"].loc[list_days_full[timestampbis]])
          count_list.append(count+1)
          timestamp += 1

    RfDf=pd.DataFrame({"Date":list_days_open_pondered,"Rf":riskfree_list,"Count":count_list})
    RfDf=RfDf.set_index("Date")
    return RfDf


def GetRiskFree(df,date,lookback):

    RfDf=GetRfDataframe(df)
    positionOfStartDate=df.index[df["Date"]==pd.to_datetime(date)][0]-lookback
    #print(positionOfStartDate)
    startDate=pd.to_datetime(df.iloc[positionOfStartDate,0])

    endDate=pd.to_datetime(date)
    RfDf=RfDf[(RfDf.index >= startDate) & (RfDf.index <= endDate )]
    CumulativeRf=[]

    for i in range(len(RfDf)):
      if i==0:
        CumulativeRf.append(pow((1+RfDf["Rf"].iloc[i]),(RfDf["Count"].iloc[i]/360)))
      else:
        CumulativeRf.append(pow((1+RfDf["Rf"].iloc[i]),(RfDf["Count"].iloc[i]/360))*CumulativeRf[i-1])

    RfDf["CumulativeRf"]=CumulativeRf
    RfDf["CumulativeRf"]= RfDf["CumulativeRf"]-1

    return RfDf["CumulativeRf"].iloc[-1]

#RiskFree=GetRiskFree(df,"2016-05-11",lookback=1800)

In [7]:
def GetWeight(df,date):
    #for the moment we will use the equal weight
    weight_vector=np.zeros((24,1))
    for i in range(0,24):
        weight_vector[i]=1/24

    return weight_vector
#Weight=GetWeight(df,"2020-05-11")


In [8]:
def GetLambda(df,date,lookback):
    returns=GetReturn(df,date,lookback)
    returns=returns+1

    avg_return=returns.prod()-1 #geometric days return
    weight_vector=GetWeight(df=0,date=0)
    Sigma=GetSigma(df,date,lookback)
    var = float((weight_vector.T @ Sigma.values @ weight_vector).item())
    lambda_value=(avg_return@weight_vector - GetRiskFree(df,date,lookback))/np.sqrt(var)
    return lambda_value


#Lambda=GetLambda(df,"2016-05-11",lookback=1800)

In [9]:
def GetPMatrix(df,date, lookback,longonly=False,proportion=3,offset=3):
    #(date)
    #print(proportion)
    #print(lookback)
    AssetColumns=["S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS"]
    bestperformer = []
    worstperformer = []
    performerc = []
    returnBestPerformer=[]
    returnWorstPerformer=[]
    endDateIndex=df.index[df["Date"]==pd.to_datetime(date)][0]
    startDateIndex=df.index[df["Date"]==pd.to_datetime(date)][0]-lookback

    for i in range(1, df.shape[1]):  #loop through asset columns
        performerc.append((((float(df.iloc[endDateIndex, i]) / float(df.iloc[startDateIndex, i]) - 1) * 100), i - 1))

    performerc.sort(reverse=True)
    for i in range(proportion):
        bestperformer.append(performerc[i][1])
        returnBestPerformer.append(performerc[i][0])

    for i in range(len(performerc) - offset - proportion, len(performerc) - offset):
        if longonly==False:
            worstperformer.append(performerc[i][1])
            returnWorstPerformer.append(performerc[i][0])

    P=np.zeros((1,24))
    if longonly==True:
        for i in range(len(AssetColumns)):
            P[0,i]=-1/(24-proportion)
    else :
        for i in range(len(AssetColumns)):
            P[0,i]=0

    for i in range(len(AssetColumns)):
        if i in bestperformer:
            P[0,i]=1/proportion
        elif i in worstperformer and longonly==False:
            P[0,i]=-1/proportion


    if len(returnWorstPerformer)==0:
        returnWorstPerformer.append(0)

    spreadLoosersWinnners=np.mean(returnBestPerformer)-np.mean(returnWorstPerformer)
    Q=np.array([[spreadLoosersWinnners/100]]) #convert to decimal
    return P, Q

#PMatrix,TempoQ=PMatrix(df,"2016-05-11",lookback=180)

In [10]:
def GetOmega(PMatrix, Sigma, c=0.99):
    #Omega is the uncertainty of the views
    factorC=(1/c-1)
    Omega=factorC*PMatrix@Sigma@np.transpose(PMatrix)


    return Omega

In [25]:
def BlackAndLittermanModel(backtestStartDate, rebalancingFrequency, lookbackPeriod, df):
    #implement the full backtest of the black and litterman model

    #---------
    #PARAMETERS
    #---------

    free_asset=0 #proportion of risk free asset allocated in the benchmark
    taux=0.01


    Sigma=GetSigma(df,backtestStartDate,lookback=lookbackPeriod)
    Lambda=GetLambda(df,backtestStartDate,lookbackPeriod)
    PMatrix,Q= GetPMatrix(df,backtestStartDate, lookback=lookbackPeriod,longonly=True,proportion=3,offset=0)
    Omega=GetOmega(PMatrix, Sigma, c=0.99)
    rf=GetRiskFree(df,backtestStartDate,lookback=lookbackPeriod)
    weights = GetWeight(df, backtestStartDate)
    weights = np.array(weights).reshape(-1, 1)
    uimplied = Lambda * (Sigma @ weights) + rf
    #BL formula
    Lambda=3





    optimizedReturn=(np.linalg.inv(np.linalg.inv(taux*Sigma)+np.transpose(PMatrix)@np.linalg.inv(Omega)@PMatrix)) @ (np.linalg.inv(taux*Sigma)@uimplied+np.transpose(PMatrix)@np.linalg.inv(Omega)@Q)




    #MarkowitzAllocation

    #WeightBL=np.linalg.inv(Sigma)@(optimizedReturn-rf)/Lambda

    from pypfopt.efficient_frontier import EfficientFrontier

    # Après Black-Litterman, tu as :
    # - bl_returns (rendements estimés)
    # - Sigma (matrice de covariance)

    # Créer l'optimiseur avec CONTRAINTES
    ef = EfficientFrontier(
        list(optimizedReturn[0]),  # tes rendements BL
        Sigma,
        weight_bounds=(-0.34, 0.34)  # min 0%, max 20% par actif
    )

    # Maximiser le ratio de Sharpe
    #WeightBL = ef.max_sharpe(risk_free_rate=0)
    WeightBL = ef.max_quadratic_utility(risk_aversion=Lambda)


    #print("BL Weights",WeightBL)
    #print("RF Weights",np.sum(WeightBL))
    weights=pd.DataFrame.from_dict(WeightBL,orient="index",columns=["Weight"])
    return weights
    pass

#print(BlackAndLittermanModel("2016-05-11", rebalancingFrequency=3, lookbackPeriod=180, df=df))






In [ ]:
from rich.console import Console
from rich.panel import Panel
from tqdm import tqdm

console = Console()

#BACK TESTER
dfbacktest=df.copy()
dfbacktest["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
dfbacktest["MonthIndex"] = dfbacktest["Date"].dt.to_period("M")

df_length = dfbacktest.shape[1] - 2  # bcs of date and spx
last_rebalance = dfbacktest.loc[0, "Date"]  # première date
month_count = 0

# 🎨 AFFICHAGE STYLÉ (sans prompts)
hold = 1
hist = 0
proportion = 3

console.print(Panel.fit(
    "[bold cyan]📊 PORTFOLIO BACKTESTER[/bold cyan]\n"
    "[dim]Black-Litterman Model[/dim]",
    border_style="cyan"
))

console.print(f"\n[yellow]⚙️  Configuration :[/yellow]")
console.print(f"   • Hold period: [cyan]{hold}[/cyan] mois")
console.print(f"   • Historique: [cyan]{hist}[/cyan] mois")
console.print(f"   • Proportion: [cyan]{proportion}[/cyan]")
console.print("\n[yellow]⏳ Lancement du backtest...[/yellow]\n")

def Backtester(df,hold, hist, proportion,df_toBL):
    #new dataframe for stock quantity
    Indexcolumns = ["Date","SPX","S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS","Money"]
    Tickercolumns=["S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS"]
    StockQty = df.copy()
    StockQty.drop(columns="MonthIndex", inplace=True)
    start=5000


    StockQty.loc[:, :] = 0
    #starting data
    MoneyAtStart = 10000000
    month_count=0
    CurrentValue=MoneyAtStart

    #first ligne
    StockQty.loc[start, "Money"] = MoneyAtStart
    StockQty.loc[start, "SPX"] = df.iloc[start, 1]
    StockQty.loc[start, "Date"] = df.iloc[start, 0]

    #start of the algorithm

    for i in tqdm(range(start,df.shape[0]), desc="Backtesting"):
      StockQty.iloc[i,0]=df.iloc[i,0]
      StockQty.iloc[i,1]=df.iloc[i,1]
      fees=0


      if df.loc[i, "Date"].month != df.loc[i-1, "Date"].month:
        month_count += 1


    # Si on atteint la période voulue
      if i>= hist and month_count % hold == 0 and df.loc[i, "Date"].month != df.loc[i - 1, "Date"].month:
        print(f"🔁 Rebalancement déclenché à la date : {df.loc[i, 'Date'].date()}")
        #print(str(df.iloc[i,0]))
        BLWeight=BlackAndLittermanModel(str(df.iloc[i,0]),3,3*22,df_toBL)
        #print(len(BLWeight))
        for index in range(len(BLWeight)):
            StockQty.iloc[i,index+2]=(BLWeight.iloc[index,0]*CurrentValue)/df.iloc[i,index+2] #qty = weight*total value/price

      else :
        for stocks in range(2,StockQty.shape[1]-1):
          StockQty.iloc[i,stocks]=StockQty.iloc[i-1,stocks] #same qty
      #value of pf

      GainOrLoss = 0
      for stocks in range(2, StockQty.shape[1]-1):
          qty = StockQty.iloc[i, stocks]
          if qty != 0.0:
            price_now = df.iloc[i, stocks]
            price_prev = df.iloc[i - 1, stocks]
            GainOrLoss += qty * (price_now - price_prev)


      CurrentValue+=GainOrLoss-fees
      StockQty.iloc[i,-1]=CurrentValue

    StockQty = StockQty.iloc[start:].reset_index(drop=True)
    return StockQty

final = Backtester(dfbacktest, hold=hold, hist=hist, proportion=proportion, df_toBL=df)

console.print("\n[green]✅ Backtest terminé avec succès ![/green]\n")

╭─────────────────────────╮
│ 📊 PORTFOLIO BACKTESTER │
│ Black-Litterman Model   │
╰─────────────────────────╯

⚙️  Configuration :

• Hold period: 1 mois

• Historique: 0 mois

• Proportion: 3

...

🔁 Rebalancement déclenché à la date : 2023-01-02


Backtesting:  61%|██████▏   | 480/783 [02:58<01:45,  2.86it/s]

🔁 Rebalancement déclenché à la date : 2023-02-01


Backtesting:  64%|██████▍   | 502/783 [03:06<01:40,  2.79it/s]

🔁 Rebalancement déclenché à la date : 2023-03-01


In [24]:
import plotly.express as px

money_norm =(final["Money"]/10000000*100)-100
spx_norm=(final["SPX"]/final["SPX"].iloc[0]* 100)-100

fix = px.line(x=final["Date"], y=[money_norm, spx_norm],labels={"value":"Évolution en %", "variable":"Série"},title="Comparaison des évolutions en %")
fix.data[0].name = "Portfolio"
fix.data[1].name = "SPX"
fix.update_layout(hovermode="x unified")

fix.show()

In [14]:
final

,Date,SPX,S5SFTW,S5PHRM,S5CPGS,S5ENRSX,S5FDBT,S5TECH,S5RETL,S5BANKX,...,S5INSU,S5FDSR,S5HOUS,S5SSEQX,S5TRAN,S5HOTR,S5CODU,S5AUCO,S5COMS,Money
0,2021-03-02 00:00:00,3870.29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,1.000000e+07
1,2021-03-03 00:00:00,3819.72,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,1.000000e+07
2,2021-03-04 00:00:00,3768.47,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,1.000000e+07
3,2021-03-05 00:00:00,3841.94,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,1.000000e+07
4,2021-03-08 00:00:00,3821.35,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,1.000000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2024-02-23 00:00:00,5088.80,-629.995029,2178.263727,2765.714715,4122.089268,3352.276754,-758.099988,-713.332932,8254.170237,...,4072.288484,4049.070765,-3241.569266,-897.534396,2748.201679,-1707.97716,6456.269909,-20459.94023,-4681.028939,7.865193e+06
779,2024-02-26 00:00:00,5069.53,-629.995029,2178.263727,2765.714715,4122.089268,3352.276754,-758.099988,-713.332932,8254.170237,...,4072.288484,4049.070765,-3241.569266,-897.534396,2748.201679,-1707.97716,6456.269909,-20459.94023,-4681.028939,7.732126e+06
780,2024-02-27 00:00:00,5078.18,-629.995029,2178.263727,2765.714715,4122.089268,3352.276754,-758.099988,-713.332932,8254.170237,...,4072.288484,4049.070765,-3241.569266,-897.534396,2748.201679,-1707.97716,6456.269909,-20459.94023,-4681.028939,7.817533e+06
781,2024-02-28 00:00:00,5069.76,-629.995029,2178.263727,2765.714715,4122.089268,3352.276754,-758.099988,-713.332932,8254.170237,...,4072.288484,4049.070765,-3241.569266,-897.534396,2748.201679,-1707.97716,6456.269909,-20459.94023,-4681.028939,7.946115e+06
